In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd

In [2]:
# Define paths to train and test directories
train_dir = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Train"
test_dir = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Test"

In [3]:
train_df = pd.DataFrame(columns=['image_path', 'label'])
test_df = pd.DataFrame(columns=['image_path', 'label'])

for label, directory in enumerate(os.listdir(train_dir)):
    for filename in os.listdir(os.path.join(train_dir, directory)):
        image_path = os.path.join(train_dir, directory, filename)
        train_df = pd.concat([train_df, pd.DataFrame({'image_path': [image_path], 'label': [label]})], ignore_index=True)

for label, directory in enumerate(os.listdir(test_dir)):
    for filename in os.listdir(os.path.join(test_dir, directory)):
        image_path = os.path.join(test_dir, directory, filename)
        test_df = pd.concat([test_df, pd.DataFrame({'image_path': [image_path], 'label': [label]})], ignore_index=True)


df = pd.concat([train_df, test_df], ignore_index=True)
del test_df,train_df
df

,image_path,label
0,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0
1,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0
2,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0
3,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0
4,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0
...,...,...
2352,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,7
2353,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,7
2354,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,8
2355,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,8


In [4]:
# Get list of directories in train_dir
labels = os.listdir(train_dir)

# Create label_map dictionary
label_map = {i: label for i, label in enumerate(labels)}
num_classes=len(label_map)
label_map

{0: 'actinic keratosis',
 1: 'basal cell carcinoma',
 2: 'dermatofibroma',
 3: 'melanoma',
 4: 'nevus',
 5: 'pigmented benign keratosis',
 6: 'seborrheic keratosis',
 7: 'squamous cell carcinoma',
 8: 'vascular lesion'}

In [5]:
max_images_per_class = 2500

# Group by label column and take first max_images_per_class rows for each group
df = df.groupby("label").apply(lambda x: x.head(max_images_per_class)).reset_index(drop=True)

C:\Users\DEVADHARSHINI K\AppData\Local\Temp\ipykernel_18892\3886418994.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("label").apply(lambda x: x.head(max_images_per_class)).reset_index(drop=True)


In [6]:
import tensorflow as tf

# Allow gpu usage
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
try:
    tf.config.experimental.set_memory_growth = True
except Exception as ex:
    print(e)

[]


In [7]:
import multiprocessing

# Get the number of CPU cores available
max_workers = multiprocessing.cpu_count()
max_workers

8

In [8]:
import concurrent.futures
from PIL import Image
# Define a function to resize image arrays
def resize_image_array(image_path):
    return np.asarray(Image.open(image_path).resize((100,75)))

# Use concurrent.futures to parallelize the resizing process
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Use executor.map to apply the function to each image path in the DataFrame
    image_arrays = list(executor.map(resize_image_array, df['image_path'].tolist()))

# Add the resized image arrays to the DataFrame
df['image'] = image_arrays
del image_arrays
df.head()

,image_path,label,image
0,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0,"[[[139, 79, 87], [138, 77, 84], [128, 63, 65],..."
1,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0,"[[[114, 85, 85], [96, 66, 60], [86, 64, 69], [..."
2,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0,"[[[166, 120, 116], [170, 126, 122], [175, 132,..."
3,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0,"[[[168, 91, 91], [179, 103, 102], [185, 112, 1..."
4,C:\Desktop\STUDY\PROJECTS\Project sem VI\archi...,0,"[[[206, 155, 156], [206, 156, 157], [211, 164,..."


In [9]:
class_counts = df['label'].value_counts().sort_index()

# Print the number of images in each class
print("Dataset Summary")
print("-" * 60)
print(f"{'Class Label':<15} {'Class Name':<30} {'Count':<10}")
print("-" * 60)
for class_label, class_name in label_map.items():
    count = class_counts[class_label]
    print(f"{class_label:<15} {class_name:<30} {count:<10}")
print("-" * 60)
print(f"{'Total':<45} {sum(class_counts):<10}")

Dataset Summary
------------------------------------------------------------
Class Label     Class Name                     Count     
------------------------------------------------------------
0               actinic keratosis              130       
1               basal cell carcinoma           392       
2               dermatofibroma                 111       
3               melanoma                       454       
4               nevus                          373       
5               pigmented benign keratosis     478       
6               seborrheic keratosis           80        
7               squamous cell carcinoma        197       
8               vascular lesion                142       
------------------------------------------------------------
Total                                         2357      


In [10]:

# Create an ImageDataGenerator object with the desired transformations
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [11]:
from sklearn.model_selection import train_test_split
features = df.drop(columns=['label','image_path'],axis=1)
target = df['label']
print(target.shape,features.shape)
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20,shuffle=True)

(2357,) (2357, 1)


In [12]:
x_train = np.asarray(x_train['image'].tolist())
x_test = np.asarray(x_test['image'].tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)
x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [13]:
from keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes = num_classes)
y_test = to_categorical(y_test,num_classes = num_classes)

In [14]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.20,shuffle=True)
# Reshape image in 3 dimensions (height = 75px, width = 100px , canal = 3)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))
y_train = y_train.astype(int)
y_validate = y_validate.astype(int)

In [15]:
img_width, img_height = 331, 331 

In [16]:
# Define paths to train and test directories
train_dir = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Train"
test_dir = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Test"

In [17]:
num_classes = len(os.listdir(train_dir))

In [18]:
batch_size = 32

In [19]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


In [20]:
# Data augmentation for test images (only rescaling)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)


In [21]:
# Generate batches of augmented data for training and validation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 2239 images belonging to 9 classes.


In [22]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 118 images belonging to 9 classes.


In [23]:
# Load NASNetLarge model without classification layers
base_model = NASNetLarge(include_top=False, input_shape=(img_width, img_height, 3), weights='imagenet')

In [24]:
# Add custom classification layers on top of NASNetLarge
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [25]:
# Combine base model with custom layers
model = Model(inputs=base_model.input, outputs=predictions)


In [26]:
# Freeze the base model layers (only train custom classification layers)
for layer in base_model.layers:
    layer.trainable = False

In [27]:
# Compile the model with specified learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=3,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size)

Epoch 1/3


C:\Users\DEVADHARSHINI K\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


69/69 ━━━━━━━━━━━━━━━━━━━━ 757s 11s/step - accuracy: 0.3115 - loss: 1.8708 - val_accuracy: 0.2708 - val_loss: 2.1354
Epoch 2/3
 1/69 ━━━━━━━━━━━━━━━━━━━━ 11:45 10s/step - accuracy: 0.3125 - loss: 1.8255

C:\Users\DEVADHARSHINI K\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


69/69 ━━━━━━━━━━━━━━━━━━━━ 18s 118ms/step - accuracy: 0.3125 - loss: 1.8255 - val_accuracy: 0.3636 - val_loss: 1.8901
Epoch 3/3
69/69 ━━━━━━━━━━━━━━━━━━━━ 751s 11s/step - accuracy: 0.4914 - loss: 1.4720 - val_accuracy: 0.3229 - val_loss: 1.9389


In [29]:
loss, accuracy = model.evaluate(test_generator, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

4/4 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step - accuracy: 0.3189 - loss: 1.9443
Test: accuracy = 0.305085  ;  loss = 1.946879


In [30]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the image
sample_img_path = r"C:\Desktop\STUDY\PROJECTS\Project sem VI\archive (1)\Skin cancer ISIC The International Skin Imaging Collaboration\Train\dermatofibroma\ISIC_0025223.jpg"
img = image.load_img(sample_img_path, target_size=(img_width, img_height))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Make predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the class label
class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class]

print("Predicted class:", predicted_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Predicted class: dermatofibroma


In [31]:
model.save('skin_disease_classification_model.h5')